# **Случайный лес**

## **Подготовка для работы в Google Colab или Kaggle**

#### Код для подключения Google Drive в Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Код для получения пути к файлам в Kaggle

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### Код для установки библиотек

In [ ]:
%pip install numpy==1.26.4 pandas==2.1.4 scikit-learn==1.7.0 matplotlib==3.8.0 seaborn==0.13.2 missingno==0.5.2

## **Важная информация**

**Для правильного воспроизведения результатов** решения задач:

* Рекомендуется придерживаться имеющего в заданиях кода в исходной последовательности. Для этого при решении задач **восстановите недостающие фрагменты кода, которые отмечены символом** `...` (Ellipsis).

* Если класс, функция или метод предусматривает параметр random_state, всегда указывайте **random_state=RANDOM_STATE**.

* Для всех параметров (кроме random_state) класса, функции или метода **используйте значения по умолчанию, если иное не указано в задании**.

**Если скорость обучения слишком низкая**, рекомендуется следующее:

* В модели или/и GridSearchCV поменяйте значение параметра n_jobs, который отвечает за параллелизм вычислений.

* Воспользуйтесь вычислительными ресурсами Google Colab или Kaggle.

***Использовать GPU не рекомендуется, поскольку результаты обучения некоторых моделей могут отличаться на CPU и GPU.***

После выполнения каждого задания **ответьте на вопросы в тесте.**

**ВНИМАНИЕ:** **После каждого нового запуска ноутбука** перед тем, как приступить к выполнению заданий, проверьте настройку виртуального окружения, выполнив код в ячейке ниже.

In [ ]:
# Код для проверки настройки виртуального окружения

import sys
from importlib.metadata import version

required = {
    'python': '3.11.x',
    'numpy': '1.26.4',
    'pandas': '2.1.4',
    'scikit-learn': '1.7.0',
    'matplotlib': '3.8.0',
    'seaborn': '0.13.2',
    'missingno': '0.5.2'
}

print(f'{"Компонент":<15} | {"Требуется":<12} | {"Установлено":<12} | {"Соответствие"}')
print('-' * 62)

environment_ok = True
for lib, req_ver in required.items():
    try:
        if lib == 'python':
            inst_ver = sys.version.split()[0]
            status = '✓' if sys.version_info.major == 3 and sys.version_info.minor == 11 else f'x (требуется {req_ver})'
        else:
            inst_ver = version(lib)
            if inst_ver == req_ver:
                status = '✓'
            else:
                environment_ok = False
                status = f'x (требуется {req_ver})'
    except:
        environment_ok = False
        inst_ver = '-'
        status = 'x (не установлена)'
    print(f'{lib:<15} | {req_ver:<12} | {inst_ver:<12} | {status}')

print('\nРезультат проверки: ', 
      '✓\nВсе версии соответствуют требованиям' 
      if environment_ok else 
      'x\nВНИМАНИЕ: Версии некоторых компонентов не соответствуют требованиям!\n'
      'Для решения проблемы обратитесь к инструкции по настройке виртуального окружения')

## **Импорт библиотек и вспомогательные функции**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.datasets import make_circles
from sklearn.preprocessing import OneHotEncoder

import missingno as msno

In [ ]:
RANDOM_STATE = 42

In [ ]:
def display_decision_boundary(classifier, features, labels):
    """
    Визуализирует границу решений классификатора на двумерных данных.

    Аргументы:
        classifier (callable): Функция или метод модели, принимающий таблицу с признаками и возвращающий предсказанные классы.
        features (pandas.DataFrame): Двумерная таблица с признаками (только два признака), по которым строится визуализация.
        labels (numpy.ndarray): Массив меток классов.
    """
    x1_min, x1_max = features.iloc[:, 0].min() - 1, features.iloc[:, 0].max() + 1
    x2_min, x2_max = features.iloc[:, 1].min() - 1, features.iloc[:, 1].max() + 1
    x1x1, x2x2 = np.meshgrid(np.arange(x1_min, x1_max, 0.01), np.arange(x2_min, x2_max, 0.01))
    decision = classifier(pd.DataFrame(np.c_[x1x1.ravel(), x2x2.ravel()], columns=features.columns)).reshape(x1x1.shape)
    plt.contourf(x1x1, x2x2, decision, alpha=0.5)
    plt.scatter(features.iloc[:, 0], features.iloc[:, 1], c=labels, edgecolors='k')
    plt.show()

## **Практическая часть**

### **Случайный лес**

Случайный лес (random forest) — это ансамблевый алгоритм машинного обучения, суть которого состоит в объединении большого ансамбля некоррелированных решающих деревьев и объединении их предсказания. По отдельности каждое дерево ансамбля показывает невысокое качество классификации, но за счёт большого их большого числа в ансамбле объединённый результат получается более точным и устойчивым к выбросам.

Случайный лес является реализацией алгоритма бэггинга (bootstrap aggregating).

**Основные принципы:**

* Бутстрэп сэмплинг (bootstrap sampling). Каждое дерево в ансамбле обучается на своей бутстрэп-подвыборке, которая формируется путём случайного выбора объектов (наблюдений) из обучающей выборки с возвращением (т.е. некоторые объекты могут повторяться несколько раз).

* Случайный выбор признаков. Для уменьшения корреляции между деревьями при на каждом шаге построения дерева (при разделении узла) рассматривается случайное подмножество признаков фиксированной мощности (например, для классификации используется $m=\sqrt{M}$, где $M$ — исходное количество признаков). Чем меньше случайное подмножество признаков, тем выше разнообразие деревьев и ниже их взаимная корреляция. 

* Агрегация предсказаний. Для классификации итоговый ответ определяется голосованием большинства деревьев (как правило, мода), для регрессии — усредненным предсказанием.

### ***Задание 1***

Сгенерируйте набор данных с двумя классами и двумя признаками с помощью [make_circles](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html) (см. код).

На обучающей выборке обучите две модели:

* `tree_circles`: DecisionTreeClassifier с неограниченной глубиной

* `rf_circles`: RandomForestClassifier c 5 деревьями (n_estimators=5)

На тестовой выборке постройте отчёты по метрикам классификации для моделей `tree` и `rf`.

In [ ]:
# Сгенерируйте набор данных с двумя классами и двумя признаками с помощью make_circles

X_circles, y_circles = make_circles(n_samples=1000, factor=0.1, noise=0.3, random_state=RANDOM_STATE)
X_circles = pd.DataFrame(X_circles)
X_circles.columns = ['x1', 'x2']

In [ ]:
# Визуализируйте датасет

plt.scatter(X_circles['x1'], X_circles['x2'], c=y_circles, edgecolors='k')
plt.show()

In [ ]:
# Разделите датасет на обучающую (60%) и тестовую (40%) выборки со стратификацией по целевой переменной
# Не забудьте зафиксировать RANDOM_STATE

X_circles_train, X_circles_test, y_circles_train, y_circles_test = ...

In [ ]:
# Обучите дерево tree_circles (DecisionTreeClassifier)
# Не забудьте зафиксировать RANDOM_STATE

tree_circles = ...

In [ ]:
# На тестовой выборке постройте отчёт по метрикам классификации для tree_circles

print(classification_report(..., digits=4))

In [ ]:
# Визуализируйте границу решений классификатора tree_circles на обучающей выборке

display_decision_boundary(tree_circles.predict, ...)

In [ ]:
# Визуализируйте границу решений классификатора tree_circles на тестовой выборке

...

In [ ]:
# Обучите случайный лес rf_circles (RandomForestClassifier) с ограничением n_estimators=5
# Не забудьте зафиксировать RANDOM_STATE

rf_circles = ...

In [ ]:
# На тестовой выборке постройте отчёт по метрикам классификации для rf_circles

...

In [ ]:
# Визуализируйте границу решений классификатора rf_circles на обучающей выборке

...

In [ ]:
# Визуализируйте границу решений классификатора rf_circles на тестовой выборке

...

### **Алгоритм построения случайного леса**

Пусть $N$ — количество наблюдений (объектов) в обучающей выборке, $M$ — количество признаков, $m$ — неполное количество случайных признаков при обучении дерева, $k$ — количество деревьев в ансамбле.

* Для каждого дерева решений из $k$ деревьев в ансамбле:

    1. Генерируется бутстрэп-подвыборка путём случайного выбора $N$ объектов из обучающей выборки с возвращением (т.е. некоторые объекты могут повторяться несколько раз).

    2. На бутстрэп-подвыборке обучается дерево решений, причём:

        * При разбиении каждого узла дерева выбирается случайное подмножество признаков мощности $m$ (случайно выбирается $m$ из $M$ признаков).

        * Лучшее разбиение узла определяется только среди выбранных $m$ признаков.

        * Дерево строится до полного исчерпания бутстрэп-подвыборки и не подвергается процедуре прунинга.

### **Датасет *Titanic Dataset***

**Для решения заданий 2 — 4 рассмотрим датасет [Titanic Dataset](https://www.kaggle.com/datasets/yasserh/titanic-dataset).**

Набор данных предназначен для предсказания шансов выживания пассажиров трагически затонувшего в 1912 году корабля "Титаник" на основе имеющейся информации о пассажирах.


Одной из особенностей датасета является наличие **пропущенных значений**.

Целевая переменная — Survived:

* 1 — пассажир выжил (целевой класс).

* 0 — пассажир погиб в катастрофе.

Датасет содержит признаки:

* PassengerId — уникальный идентификатор пассажира.

* Pclass — класс билета (1, 2 или 3).

* Name — имя пассажира.

* Sex — пол пассажира.

* Age — возраст пассажира.

* SibSp — количество братьев, сестёр или супругов на борту.

* Parch — количество родителей или детей на борту.

* Ticket — номер билета.

* Fare — стоимость билета.

* Cabin — номер каюты.

* Embarked — порт посадки (C = Cherbourg, Q = Queenstown, S = Southampton).

### ***Задание 2***

Выполните предобработку датасета (см. код).

Используя обучающую выборку, сгенерируйте 5 бутстрэп-подвыброк ($k=5$), используя функцию [choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) (путём случайного выбора индексов датасета `X_titanic_train` с повторениями).

На каждой из сгенерированных подвыборок обучите дерево DecisionTreeClassifier с неограниченной глубиной и с параметрами:

* **random_state=i**, где $i=0,...,4$ — индекс дерева (индекс подвыборки)

* **max_features=5** — параметр, задающий максимальное число случайных признаков, которое рассматривается для наилучшего разбиения в каждом узле дерева.

Используя обученные деревья решений, предскажите вероятности выживания пассажиров **в тестовой выборке** (запишите предсказанные вероятности в `titanic_trees_y_test_proba`) и посчитайте AUC для предсказания каждого из деревьев решений в отдельности.

Рассчитайте среднее значение AUC, если прогноз на тестовой выборке осуществляется каждым деревом по отдельности.

Рассматривая обученные деревья как ансамбль, постройте прогноз вероятностей для этого ансамбля (случайного леса) **на тестовой выборке** — **усредните (найдите среднее) значения вероятностей выживания**, предсказанные деревьями, входящими в ансамбль.

Рассчитайте значение AUC для прогноза ансамбля (случайного леса) на тестовой выборке.

Обучите модель случайного леса `rf_titanic_5` (RandomForestClassifier) c параметром n_estimators=5 (5 деревьев). 

Рассчитайте AUC модели `rf_titanic_5` на **тестовой выборке**.

Сравните значения AUC модели `rf_titanic_5`, среднего AUC для деревьев и AUC ансамбля (случайного леса).

In [ ]:
# Считайте набор данных

df_titanic = pd.read_csv('titanic.csv')
df_titanic

In [ ]:
# Удалите колонки, не несущие существенной информации: PassengerId, Name, Ticket, Cabin

df_titanic = df_titanic.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

In [ ]:
# Убедитесь, что в датасете присутствуют пропуски в данных

...

In [ ]:
# Посчитайте доли пропусков во всей выборке

titanic_nan_info = pd.DataFrame({'NaN share': ...})
titanic_nan_info = titanic_nan_info[titanic_nan_info['NaN share'] != 0.0]

In [ ]:
# Выполните визуальный анализ пропущенных значений во всей выборке с помощью msno.bar

...

In [ ]:
# Выполните визуальный анализ пропущенных значений в обучающей выборке с помощью msno.matrix

...

In [ ]:
# Создайте список категориальных переменных (не включая целевую переменную)

titanic_cat_feat = ['Pclass', 'Sex', 'Embarked']

In [ ]:
# Выделите объясняемый фактор в отдельную переменную

X_titanic, y_titanic = ...

In [ ]:
# Закодируйте категориальные признаки числами 0 и 1 с помощью OneHotEncoder

titanic_encoder = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

X_titanic_encoded = ...
X_titanic = ...

In [ ]:
# Разделите датасет на обучающую (60%) и тестовую (40%) выборки со стратификацией по целевой переменной
# Не забудьте зафиксировать RANDOM_STATE

X_titanic_train, X_titanic_test, y_titanic_train, y_titanic_test = ...

In [ ]:
# Используя обучающую выборку, сгенерируйте 5 бутстрэп-подвыброк, используя функцию choice

# Не забудьте зафиксировать RANDOM_STATE (с помощью переменной rng)
rng = np.random.RandomState(RANDOM_STATE)

k = 5 # Число деревьев в ансамбле (число бутстрэп-подвыброк)

train_N = ... # Размер обучающей выборки

train_idx = list(...) # Список с индексами наблюдений обучающей выборки

# Выполните случайный выбор индексов train_idx с повторениями
titanic_bootstraps = [rng.choice(..., size=..., replace=...) for _ in range(k)]

In [ ]:
# На каждой из бутстрэп-подвыборок обучите дерево DecisionTreeClassifier с неограниченной глубиной и с параметрами
#   random_state=i
#   max_features=5
# Для каждого дерева используйте отдельную бутстрэп-подвыборку
# После построения каждого дерева предскажите вероятности выживания пассажиров в тестовой выборке и посчитайте AUC

titanic_trees_y_test_proba = [] # Список для записи векторов предсказаний деревьев
titanic_trees_roc_auc = [] # Список для записи AUC деревьев

for i in range(k):

    X_titanic_fold_train = ...
    y_titanic_fold_train = y_titanic_train[X_titanic_fold_train.index]

    tree_titanic_fold = DecisionTreeClassifier(...).fit(...)

    y_test_proba_tree_titanic_fold = ...
    titanic_trees_y_test_proba.append(y_test_proba_tree_titanic_fold)

    roc_auc_tree_titanic_fold = ...
    titanic_trees_roc_auc.append(roc_auc_tree_titanic_fold)

    print('AUC дерева {}: {:.4f}'.format(i, ...))

titanic_trees_y_proba = np.array(titanic_trees_y_test_proba)

In [ ]:
# Удостоверьтесь, что размерность массива titanic_trees_y_proba — (5, 357)

titanic_trees_y_proba.shape

In [ ]:
# Рассчитайте среднее значение AUC, если прогноз на тестовой выборке осуществляется каждым деревом по отдельности

...

In [ ]:
# Посчитайте прогноз случайного леса как усреднённое значение вероятностей, предсказанных деревьями ансамбля

y_test_proba_forest_titanic = ...

In [ ]:
# Рассчитайте значение AUC для прогноза ансамбля (случайного леса) на тестовой выборке

...

In [ ]:
# Обучите модель RandomForestClassifier c параметром n_estimators=5
# Не забудьте зафиксировать RANDOM_STATE

rf_titanic_5 = ...

In [ ]:
# Рассчитайте AUC модели rf_titanic_5 на тестовой выборке

...

### **RandomizedSearchCV**

[RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) — метод подбора оптимальных гиперпараметров модели, суть которого состоит в переборе случайных комбинаций значений параметров из заданных диапазонов или распределений вместо полного перебора всех возможных комбинаций (как это реализовано в GridSearchCV).

**RandomizedSearchCV и GridSearchCV**:

* **RandomizedSearchCV** выбирает комбинации случайно. Это снижает вычислительные затраты и позволяет находить приемлемые (иногда лучшие) гиперпараметры при меньших вычислительных затратах. **Параметр n_iter** указывает число случайных комбинаций гиперпараметров, которые будут протестированы в процессе подбора.

* **GridSearchCV** перебирает все комбинации значений гиперпараметров. Это гарантирует нахождение результата внутри сетки, но требует много времени, особенно при большом числе параметров или возможных значений.

Подробнее можно изучить по **ссылке:**

* [Рандомизированная оптимизация параметров | scikit-learn.ru](https://scikit-learn.ru/stable/modules/grid_search.html#randomized-parameter-search)

### ***Задание 3***

**ВНИМАНИЕ:** Для решения этого задания используйте:

* Обучающую и тестовую выборку из задания 2: `X_titanic_train`, `X_titanic_test`, `y_titanic_train`, `y_titanic_test`.

На обучающей выборке обучите две модели, предварительно подобрав оптимальные значения гиперпараметров обучения **с помощью RandomizedSearchCV** (n_iter=50 — 50 итераций):

* `tree_titanic` — дерево DecisionTreeClassifier.

* `rf_titanic` — случайный лес RandomForestClassifier.

Выведите оптимальные гиперпараметры обучения моделей `tree_titanic` и `rf_titanic` и номера итераций, на которых были достигнуты оптимальные комбинации параметров в рамках подбора с помощью RandomizedSearchCV.

Для моделей `tree_titanic` и `rf_titanic` **на тестовой выборке** постройте отчёт по метрикам классификации и рассчитайте значения AUC.

In [ ]:
# Обучите модель tree_titanic с оптимальными гиперпараметрами
# Оптимальные гиперпараметры обучения подберите с помощью RandomizedSearchCV
# Не забудьте зафиксировать RANDOM_STATE

params = {
    'max_depth': range(5, 15),
    'min_samples_split': range(2, 10),
    'min_samples_leaf': range(2, 10),
}
n_iter = 50
scoring = 'roc_auc'
cv = 5

cv_tree_titanic = RandomizedSearchCV(
    estimator=...,
    param_distributions=...,
    n_iter=...,
    scoring=...,
    cv=...,
    random_state=RANDOM_STATE
).fit(...)

tree_titanic = ...

In [ ]:
# Выведите оптимальные гиперпараметры обучения tree_titanic и номер итерации, на котором они были достигнуты

print(f'Оптимальные параметры DecisionTreeClassifier на итерации {...}: {...}')

In [ ]:
# Постройте отчёт по метрикам классификации для модели tree_titanic на тестовой выборке

...

In [ ]:
# Рассчитайте значение метрики AUC для модели tree_titanic на тестовой выборке

...

In [ ]:
# Обучите модель rf_titanic с оптимальными гиперпараметрами
# Оптимальные гиперпараметры обучения подберите с помощью RandomizedSearchCV
# Не забудьте зафиксировать RANDOM_STATE

params = {
    'n_estimators': range(5, 200),
    'min_samples_split': range(2, 10),
    'min_samples_leaf': range(2, 10),
}
n_iter = 50
scoring = 'roc_auc'
cv = 5

cv_rf_titanic = RandomizedSearchCV(
    estimator=...,
    param_distributions=...,
    n_iter=...,
    scoring=...,
    cv=...,
    random_state=RANDOM_STATE
).fit(...)

rf_titanic = ...

In [ ]:
# Выведите оптимальные гиперпараметры обучения rf_titanic и номер итерации, на котором они были достигнуты

...

In [ ]:
# Постройте отчёт по метрикам классификации для модели rf_titanic на тестовой выборке

...

In [ ]:
# Рассчитайте значение метрики AUC для модели rf_titanic на тестовой выборке

...

### **Isolation Forest**

Isolation Forest — это алгоритм машинного обучения без учителя (unsupervised machine learning), предназначенный для обнаружения аномалий. В отличие от методов, основанных на расстояниях или плотностях, Isolation Forest **использует ансамбль решающих деревьев** для изоляции аномальных точек. Ключевая идея этого алгоритма состоит в том, что аномальные наблюдения можно "изолировать" за меньшее число шагов, чем "обычные" данные, поскольку аномальные наблюдения встречаются редко и имеют необычные значения признаков.

Для определения аномальных значений в Isolation Forest используется **оценка аномальности**:

$$s(x,n)=2^{-\frac{\mathbb{E}(h(x))}{c(x)}}$$

где $\mathbb{E}(h(x))$ — средняя глубина (число шагов до изоляции наблюдения) по всем деревьям, $c(x)$ — нормирующая константа (зависит от размера данных).

Подробнее можно изучить по **ссылке:**

* [How to perform anomaly detection with the Isolation Forest algorithm | towardsdatascience.com](https://towardsdatascience.com/how-to-perform-anomaly-detection-with-the-isolation-forest-algorithm-e8c8372520bc/)

### **Датасет *Credit Card Fraud Detection***

**Для решения задания 4 рассмотрим датасет [Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).**

Датасет предназначен для обнаружения мошеннических операций (фрод-операций) с кредитными картами. В нем собраны данные по транзакциям, совершенным европейскими владельцами карт за два дня в сентябре 2013 года. Данные о транзакциях представлены в виде числовых переменных, полученных в результате преобразования методом главных компонент (PCA). 

Целевая переменная — Class (метка мошеннической операции):

* 1 — фрод-транзакция (аномальное значение).

* 0 — транзакция не является мошеннической.

Датасет характеризуется сильным дисбалансом классов (мошенничество — очень редкое событие): среди 284807 транзакций было выявлено 492 фрод-операций (0.172% от всех транзакций). 

### ***Задание 4***

Обучите модель `if_fraud` (IsolationForest) **на всей выборке** c 500 деревьями в ансамбле и определите аномальные наблюдения в выборке.

Рассматривая задачу как задачу классификации, где **аномальное значение соответствует случаю мошенничества (Class = 1)**, на всей выборке для модели `if_fraud` постройте матрицу ошибок и отчёт по метрикам классификации. Рассчитайте долю мошеннических транзакций, которую смогла выявить модель.

**ВНИМАНИЕ:** Поскольку алгоритм Isolation Forest относится к классу задач машинного обучения без учителя, при обучении модели метки классов (вектор истинных значений объясняемой переменной) не используются. Метки классов нужны в рамках данной задачи с целью оценки того, насколько точно модель Isolation Forest смогла выявить мошеннические операции среди всех транзакций.

*Важным параметром в модели IsolationForest является параметр contamination. Он определяет **ожидаемую долю аномалий в данных**, то есть то, сколько процентов объектов модель должна считать "аномальными" после обучения. В рамках данного задания рассматривается стандартное значение contamination.*

In [ ]:
# Считайте набор данных

df_fraud = pd.read_csv('creditcard.csv')
df_fraud

In [ ]:
# Выделите объясняемый фактор в отдельную переменную

X_fraud, y_fraud = ...

In [ ]:
# Обучите модель if_fraud с 500 деревьями в ансамбле
# Метки классов y_fraud не используются при обучении
# Не забудьте зафиксировать RANDOM_STATE

if_fraud = ...

In [ ]:
# С помощью модели if_fraud определите аномальные наблюдения
#   1 — не аномальное наблюдение
#   -1 — аномальное наблюдение

pred_if_fraud = ...
np.unique(pred_if_fraud, return_counts=True)

In [ ]:
# Перейдите от прогноза аномальных наблюдений к целевой переменной:
#   1 (не аномальное наблюдение) —> 0 (обычная транзакция)
#   -1 (аномальное наблюдение) —> 1 (мошенническая транзакция)

y_pred_if_fraud = ...

In [ ]:
# Постройте матрицу ошибок

cf_matrix = ...
plt.figure(figsize=(6, 4))
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
ax.set_title('Confusion Matrix\n')
ax.set_xlabel('Predicted label')
ax.set_ylabel('True label')
plt.show()

In [ ]:
# Постройте отчёт по метрикам классификации

...

In [ ]:
# Рассчитайте долю мошеннических транзакций, которую смогла выявить модель.

...